In [11]:
%pip install keras
%pip install torch
%pip install torchvision


[notice] A new release of pip is available: 23.2.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 23.2.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 23.2.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [13]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from torchvision import transforms
from keras.datasets import mnist

In [14]:
# Define PyTorch Model
class MNISTNet(nn.Module):
    def __init__(self):
        super(MNISTNet, self).__init__()
        self.fc1 = nn.Linear(784, 512)
        self.fc2 = nn.Linear(512, 512)
        self.fc3 = nn.Linear(512, 10)
        self.dropout = nn.Dropout(0.2)
        self.relu = nn.ReLU()
        
    def forward(self, x):
        x = x.view(-1, 784)  # Flatten input
        x = self.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.relu(self.fc2(x))
        x = self.dropout(x)
        x = self.fc3(x)
        return x

# Initialize model
model = MNISTNet()

In [15]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()

In [16]:
# Convert numpy arrays to PyTorch tensors
X_train_tensor = torch.from_numpy(X_train).float()
y_train_tensor = torch.from_numpy(y_train).long()
X_test_tensor = torch.from_numpy(X_test).float()
y_test_tensor = torch.from_numpy(y_test).long()

# Create DataLoader
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=128)

In [17]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters())

for epoch in range(5):
    model.train()
    train_loss = 0
    correct = 0
    
    for data, target in train_loader:
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item()
        pred = output.argmax(dim=1)
        correct += pred.eq(target).sum().item()
    
    print(f'Epoch {epoch+1} - Train Accuracy: {100.*correct/len(train_loader.dataset):.2f}%')

Epoch 1 - Train Accuracy: 90.02%
Epoch 2 - Train Accuracy: 94.88%
Epoch 3 - Train Accuracy: 95.71%
Epoch 4 - Train Accuracy: 96.03%
Epoch 5 - Train Accuracy: 96.17%


In [18]:
# Save PyTorch model state dict
torch.save(model.state_dict(), 'mnist_model.pth')

In [19]:
def predict_from_image(image_path, model):
    try:
        img = Image.open(image_path).convert('L')
        img = img.resize((28, 28))
        transform = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize((0.5,), (0.5,))
        ])
        img_tensor = transform(img).unsqueeze(0)
        
        with torch.no_grad():
            output = model(img_tensor)
            prediction = output.argmax().item()
            
        return prediction
    except Exception as e:
        print(f"Error: {e}")
        return None

In [20]:
from PIL import Image

# Load the PyTorch model state dict
model = MNISTNet()
model.load_state_dict(torch.load('mnist_model.pth'))
model.eval()

# Predict from image
predicted_class = predict_from_image('Digits/sample_digit_1.png', model)
if predicted_class is not None:
    print(f"The predicted class for the image is: {predicted_class}")

The predicted class for the image is: 8
